## DuckDB to BQ + Polars

In [8]:
import duckdb
from google.cloud import bigquery
from google.oauth2 import service_account

key_path = "service_intro-bq-mio.json"

project_id = "intro-bq-01"
database = "windsom_pets"
table = "sales"

credentials = service_account.Credentials.from_service_account_file(key_path)
bqclient = bigquery.Client(credentials=credentials, project=credentials.project_id)
selected_table = bigquery.TableReference.from_string(f"{project_id}.{database}.{table}")
rows = bqclient.list_rows(selected_table)
bq_data = rows.to_arrow(create_bqstorage_client=True)

conn = duckdb.connect()

query = """ SELECT * 
            FROM bq_data 
            --LIMIT 5
        """

df = conn.sql(query).pl()
df

customer_id,transaction_type,transaction_id,transaction_date,product_id,quantity,total_amount
i64,str,i64,date,i64,i64,f64
69434132,"""Retail""",70871315,2020-02-19,100041,1,6.0
69434132,"""Retail""",69435751,2023-05-17,100041,1,6.0
69434132,"""Retail""",56763934,2021-12-24,100041,1,6.0
69434132,"""Retail""",25561811,2022-09-08,100041,1,6.0
69434132,"""Retail""",37754434,2019-12-31,100041,1,6.0
…,…,…,…,…,…,…
30571760,"""Wholesale""",80687092,2020-07-21,100054,500,26250.0
30571760,"""Wholesale""",15067315,2021-01-11,100054,500,26250.0
30571760,"""Wholesale""",84109115,2021-02-16,100054,500,26250.0


In [ ]:
df.write_parquet('bq_data.parquet') # polar df

## Read Parquet from DuckDB to Pandas

In [12]:
import duckdb

con = duckdb.connect()

path_sa = 'bq_data.parquet'
con.execute(f"CREATE TABLE data AS SELECT * FROM read_parquet({path_sa})")
df1 = con.execute("SELECT * FROM data").fetchdf()
df1

,customer_id,transaction_type,transaction_id,transaction_date,product_id,quantity,total_amount
0,69434132.0,Retail,70871315,2020-02-19,100041,1,6.0
1,69434132.0,Retail,69435751,2023-05-17,100041,1,6.0
2,69434132.0,Retail,56763934,2021-12-24,100041,1,6.0
3,69434132.0,Retail,25561811,2022-09-08,100041,1,6.0
4,69434132.0,Retail,37754434,2019-12-31,100041,1,6.0
...,...,...,...,...,...,...,...
217102,30571760.0,Wholesale,80687092,2020-07-21,100054,500,26250.0
217103,30571760.0,Wholesale,15067315,2021-01-11,100054,500,26250.0
217104,30571760.0,Wholesale,84109115,2021-02-16,100054,500,26250.0
217105,30571760.0,Wholesale,61732974,2021-01-30,100054,500,26250.0


## Read Parquet from DuckDB to Polars

In [24]:
import duckdb
import polars as pl

con = duckdb.connect()

path_sa = 'bq_data.parquet'

con.execute(f"CREATE TABLE data AS SELECT * FROM read_parquet({path_sa})")
arrow_table = con.execute("SELECT * FROM data").arrow()
df = pl.LazyFrame(arrow_table).collect()
df

customer_id,transaction_type,transaction_id,transaction_date,product_id,quantity,total_amount
i64,str,i64,date,i64,i64,f64
69434132,"""Retail""",70871315,2020-02-19,100041,1,6.0
69434132,"""Retail""",69435751,2023-05-17,100041,1,6.0
69434132,"""Retail""",56763934,2021-12-24,100041,1,6.0
69434132,"""Retail""",25561811,2022-09-08,100041,1,6.0
69434132,"""Retail""",37754434,2019-12-31,100041,1,6.0
…,…,…,…,…,…,…
30571760,"""Wholesale""",80687092,2020-07-21,100054,500,26250.0
30571760,"""Wholesale""",15067315,2021-01-11,100054,500,26250.0
30571760,"""Wholesale""",84109115,2021-02-16,100054,500,26250.0


## to write on BigQuery with DuckDB

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account

key_path = "service_intro-bq-mio.json"

project_id = "intro-bq-01"
database = "windsom_pets"
table = "sales"

credentials = service_account.Credentials.from_service_account_file(key_path)
bqclient = bigquery.Client(credentials=credentials, project=credentials.project_id)
table_ref = bqclient.dataset(database).table(table)

job_config = bigquery.LoadJobConfig()
job_config.autodetect = True
job_config.source_format = bigquery.SourceFormat.PARQUET

job = bqclient.load_table_from_dataframe(df1, table_ref, job_config=job_config)
job.result()
print(f"Loaded {job.output_rows} rows into {database}:{table}")

Loaded 217107 rows into windsom_pets:sales
